# 🫀 LSTM Autoencoder para Detección de Anomalías en Series Temporales

Este notebook implementa un **LSTM Autoencoder** para detección de anomalías en series temporales (por ejemplo, señales ECG).

**Características principales:**
- Entrenamiento no supervisado (solo con ejemplos normales)
- Validación y test con etiquetas para evaluación
- Integración con MLflow para tracking de experimentos
- Orquestación con Prefect 2.x
- Soporte automático para GPU (RTX 5080 compatible)

> ⚠️ **IMPORTANTE EN WINDOWS:** Ejecuta la celda de **Setup DLLs CUDA** (celda 3) **ANTES** de la celda de imports. Esto es necesario para que PyTorch pueda cargar las DLLs de CUDA correctamente.

> ▶️ **Instrucciones:** 
> 1. Ejecuta la celda de **Setup DLLs CUDA** primero
> 2. Luego ejecuta todas las demás celdas en orden
> 3. Completa la función `load_raw_data()` con tu lógica de carga de datos


## 📋 Índice

1. **Configuración general** - Imports, semillas, dispositivo, hiperparámetros
2. **Carga y preparación de datos** - Funciones para cargar y preparar datos
3. **Definición del modelo LSTM Autoencoder** - Arquitectura del modelo
4. **Funciones de entrenamiento y evaluación** - Loops de entrenamiento y validación
5. **Integración con MLflow** - Configuración y logging
6. **Orquestación con Prefect** - Flujo principal con Prefect
7. **Ejecución del flujo completo** - Celda final para ejecutar todo


---

## 1. ⚙️ Configuración General


In [208]:
# ========================================
# 🔧 Setup RTX 5080 — dependencias + CUDA DLL
# Ejecuta una sola vez (o tras actualizar drivers/librerías)
# ========================================
import os
import sys
import subprocess
from pathlib import Path
from textwrap import dedent

print(f"Python: {sys.executable}")
print(f"Working dir: {Path.cwd().resolve()}")

# Rutas candidatas para DLLs de CUDA
CUDA_CANDIDATES = [
    os.environ.get("CUDA_PATH"),
    os.environ.get("CUDA_PATH_V12_8"),
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8",
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin",
    r"C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\libnvvp",
    r"C:\Program Files\NVIDIA\CUDNN",
]

# Añadir rutas DLL en Windows (necesario antes de importar torch)
added = []
if hasattr(os, "add_dll_directory"):
    for candidate in CUDA_CANDIDATES:
        if not candidate:
            continue
        path = Path(candidate)
        if path.is_dir():
            try:
                os.add_dll_directory(str(path))
                added.append(str(path))
            except (FileNotFoundError, OSError):
                pass

if added:
    print("DLL directories añadidos:")
    for path in added:
        print(f"  - {path}")

# Instalar dependencias base si no están instaladas
BASE_PACKAGES = [
    "mlflow>=2.16",
    "prefect>=3",
    "scikit-learn",
    "matplotlib",
    "pandas",
    "numpy",
]

def pip_install(spec: str) -> None:
    module_name = spec.split("==")[0].split("[")[0].replace("-", "_")
    try:
        __import__(module_name)
        print(f"✔ {spec} ya instalado")
    except Exception:
        print(f"⏳ Instalando {spec} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", spec])

for pkg in BASE_PACKAGES:
    pip_install(pkg)

# Comando para instalar PyTorch nightly con CUDA 12.8 (para RTX 5080)
TORCH_INSTALL_CMD = [
    sys.executable,
    "-m",
    "pip",
    "install",
    "--upgrade",
    "--pre",
    "torch",
    "torchvision",
    "torchaudio",
    "--index-url",
    "https://download.pytorch.org/whl/nightly/cu128",
]

def ensure_torch_cuda() -> "tuple[object | None, dict]":
    """Importa torch, o instala la nightly cu128 si hace falta."""
    info: dict[str, str | float | bool] = {}
    try:
        import torch  # type: ignore
        info["torch_version"] = getattr(torch, "__version__", "desconocida")
        info["cuda_version"] = getattr(getattr(torch, "version", object()), "cuda", "desconocida")
        info["cuda_available"] = bool(torch.cuda.is_available())
        if "cu128" not in info["torch_version"] and not str(info["cuda_version"]).startswith("12.8"):
            raise RuntimeError(
                f"Build {info['torch_version']} no es cu128. Se reinstalará la nightly para RTX 5080."
            )
        return torch, info
    except Exception as err:
        print("⚠️ Torch no usable todavía:", err)
        print("   Desinstalando PyTorch corrupto...")
        # Desinstalar primero
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "torch", "torchvision", "torchaudio"])
        print("   Instalando nightly cu128 desde PyTorch (puede tardar).")
        subprocess.check_call(TORCH_INSTALL_CMD)
        print("\n" + "="*60)
        print("⚠️ IMPORTANTE: PyTorch fue reinstalado.")
        print("   DEBES REINICIAR EL KERNEL DE JUPYTER ahora:")
        print("   Kernel → Restart Kernel")
        print("   Luego ejecuta esta celda de nuevo.")
        print("="*60)
        # Intentar importar de todas formas (puede fallar, pero al menos intentamos)
        import importlib
        import time
        time.sleep(2)
        importlib.invalidate_caches()
        try:
            import torch  # type: ignore
            info["torch_version"] = getattr(torch, "__version__", "desconocida")
            info["cuda_version"] = getattr(getattr(torch, "version", object()), "cuda", "desconocida")
            info["cuda_available"] = bool(torch.cuda.is_available())
            return torch, info
        except Exception as e2:
            print(f"\n❌ No se pudo importar PyTorch después de reinstalar: {e2}")
            print("   Por favor, REINICIA EL KERNEL y ejecuta esta celda de nuevo.")
            raise RuntimeError("Reinicia el kernel de Jupyter y ejecuta esta celda de nuevo.") from e2

# Intentar importar/instalar PyTorch
torch, torch_info = ensure_torch_cuda()

print("\nTorch info:")
for k, v in torch_info.items():
    print(f"  - {k}: {v}")

if torch_info.get("cuda_available"):
    try:
        gpu_name = torch.cuda.get_device_name(0)
        cc = torch.cuda.get_device_properties(0)
        print(f"GPU detectada: {gpu_name} | SM {cc.major}{cc.minor}")
    except Exception as e:
        print("⚠️ CUDA disponible pero no se pudo consultar GPU:", e)
else:
    print(dedent(
        """
        ⚠️ CUDA sigue inactiva. Revisa drivers / reinicia kernel tras la instalación.
        Si el problema continúa, ejecuta manualmente:
          pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
        """
    ))


Python: c:\Python311\python.exe
Working dir: S:\Proyecto final\Books
DLL directories añadidos:
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\bin
  - C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.8\libnvvp
⏳ Instalando mlflow>=2.16 ...
⏳ Instalando prefect>=3 ...
⏳ Instalando scikit-learn ...
✔ matplotlib ya instalado
✔ pandas ya instalado
✔ numpy ya instalado

Torch info:
  - torch_version: 2.10.0.dev20251121+cu128
  - cuda_version: 12.8
  - cuda_available: True
GPU detectada: NVIDIA GeForce RTX 5080 | SM 120


In [209]:
# ========================================
# Imports y dependencias
# ========================================
# ⚠️ IMPORTANTE: Ejecuta la celda anterior (Setup DLLs) antes de esta celda
# torch ya está importado en la celda anterior
import random
import json
import time
from pathlib import Path
from typing import Tuple, Dict, List, Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# torch ya está importado en la celda anterior, solo importamos los submódulos
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    accuracy_score,
    classification_report,
)
import mlflow
import mlflow.pytorch
from prefect import task, flow
from prefect.tasks import NO_CACHE

print("✓ Todos los imports completados")


✓ Todos los imports completados


In [210]:
# ========================================
# DIAGNÓSTICO: Verificar datos después de cargar
# ========================================
def diagnosticar_datos(
    X_train_norm: np.ndarray,
    X_val_norm: np.ndarray,
    X_test_norm: np.ndarray,
    nombre: str = "DATOS RAW"
):
    """Función para diagnosticar el estado de los datos."""
    print("\n" + "="*70)
    print(f"🔍 DIAGNÓSTICO: {nombre}")
    print("="*70)
    
    for nombre_arr, arr in [
        ("X_train_norm", X_train_norm),
        ("X_val_norm", X_val_norm),
        ("X_test_norm", X_test_norm),
    ]:
        if len(arr) == 0:
            print(f"\n{nombre_arr}: Array vacío")
            continue
            
        print(f"\n{nombre_arr} (shape: {arr.shape}):")
        print(f"  - Min: {arr.min():.6f}, Max: {arr.max():.6f}")
        print(f"  - Mean: {arr.mean():.6f}, Std: {arr.std():.6f}")
        
        # Solo mostrar valores únicos si no son demasiados
        unique_vals = np.unique(arr.flatten())
        if len(unique_vals) <= 10:
            print(f"  - Valores únicos: {unique_vals[:10]}")
        else:
            print(f"  - Valores únicos (primeros 5): {unique_vals[:5]}")
        
        n_zeros = (arr == 0).sum()
        n_nan = np.isnan(arr).sum()
        n_inf = np.isinf(arr).sum()
        total = arr.size
        
        print(f"  - Zeros: {n_zeros:,} ({100*n_zeros/total:.2f}%)")
        print(f"  - NaN: {n_nan:,} ({100*n_nan/total:.2f}%)")
        print(f"  - Inf: {n_inf:,} ({100*n_inf/total:.2f}%)")
        
        if arr.std() < 1e-6:
            print(f"  ⚠ ADVERTENCIA: Std muy pequeño ({arr.std():.6e}), datos podrían estar constantes")
        if n_zeros == total:
            print(f"  ❌ ERROR: Todos los valores son cero!")
        if arr.max() == 0 and arr.min() == 0:
            print(f"  ❌ ERROR: Array completamente en cero!")
        if arr.std() == 0:
            print(f"  ❌ ERROR: Desviación estándar es cero! Datos constantes!")
    
    print("="*70)


# ========================================
# Función mejorada de limpieza de datos
# ========================================
def clean_data_smart(X: np.ndarray, fill_value: str = "mean") -> np.ndarray:
    """
    Limpia NaN e Inf de forma inteligente.
    
    Args:
        X: Array a limpiar
        fill_value: Qué usar para reemplazar NaN/Inf ("mean", "median", o "zero")
    
    Returns:
        Array limpio
    """
    X = X.copy()  # No modificar el original
    
    # Verificar si hay NaN o Inf
    has_nan = np.any(np.isnan(X))
    has_inf = np.any(np.isinf(X))
    
    if not (has_nan or has_inf):
        return X
    
    n_nan = np.isnan(X).sum()
    n_inf = np.isinf(X).sum()
    
    print(f"    ⚠ Encontrados {n_nan} NaN y {n_inf} Inf")
    
    # Calcular valor de reemplazo
    if fill_value == "mean":
        # Calcular media solo de valores válidos
        valid_mask = np.isfinite(X)
        if valid_mask.any():
            fill_val = np.mean(X[valid_mask])
            print(f"    → Reemplazando con media de valores válidos: {fill_val:.6f}")
        else:
            fill_val = 0.0
            print(f"    ⚠ No hay valores válidos, usando 0")
    elif fill_value == "median":
        valid_mask = np.isfinite(X)
        if valid_mask.any():
            fill_val = np.median(X[valid_mask])
            print(f"    → Reemplazando con mediana de valores válidos: {fill_val:.6f}")
        else:
            fill_val = 0.0
            print(f"    ⚠ No hay valores válidos, usando 0")
    else:  # "zero"
        fill_val = 0.0
        print(f"    → Reemplazando con 0")
    
    # Reemplazar NaN e Inf
    X = np.nan_to_num(X, nan=fill_val, posinf=fill_val, neginf=fill_val)
    
    return X


In [211]:
# ========================================
# Configuración de semillas aleatorias
# ========================================
def set_seed_everywhere(seed: int = 42) -> None:
    """Fija semillas para reproducibilidad."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

SEED = 42
set_seed_everywhere(SEED)
print(f"✓ Semilla fijada: {SEED}")


✓ Semilla fijada: 42


In [212]:
# ========================================
# Configuración de dispositivo (GPU/CPU)
# ========================================
def get_device() -> torch.device:
    """Detecta y configura el dispositivo (GPU si está disponible)."""
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✓ GPU detectada: {gpu_name}")
        print(f"  CUDA Version: {torch.version.cuda}")
        print(f"  PyTorch Version: {torch.__version__}")
    else:
        device = torch.device("cpu")
        print("⚠ GPU no disponible, usando CPU")
    return device

DEVICE = get_device()
print(f"Dispositivo seleccionado: {DEVICE}")


✓ GPU detectada: NVIDIA GeForce RTX 5080
  CUDA Version: 12.8
  PyTorch Version: 2.10.0.dev20251121+cu128
Dispositivo seleccionado: cuda


In [213]:
# ========================================
# ⚙️ CONFIGURACIÓN DE HIPERPARÁMETROS
# ========================================
# Modifica estos valores para ajustar la arquitectura y entrenamiento

CONFIG = {
    # ===== ARQUITECTURA DEL MODELO =====
    "INPUT_SIZE": 3,  # Dimensión de entrada por timestep (ej: 3 derivaciones ECG)
    "HIDDEN_SIZES_ENCODER": [64, 32],  # Neuronas por capa LSTM del encoder
    "HIDDEN_SIZES_DECODER": [32, 64],  # Neuronas por capa LSTM del decoder
    "LATENT_DIM": 16,  # Dimensión del espacio latente
    "NUM_LAYERS": 2,  # Número de capas LSTM (debe coincidir con longitudes de HIDDEN_SIZES)
    "DROPOUT": 0.1,  # Dropout en capas LSTM
    
    # ===== ENTRENAMIENTO =====
    "EPOCHS": 30,
    "BATCH_SIZE": 64,
    "LEARNING_RATE": 0.001,
    "WEIGHT_DECAY": 1e-5,
    "CLIP_GRAD_NORM": 1.0,  # None para desactivar gradient clipping
    
    # ===== DETECCIÓN DE ANOMALÍAS =====
    "THRESHOLD_PERCENTILE": 95.0,  # Percentil para calcular umbral de anomalías
    
    # ===== MLFLOW =====
    "MLFLOW_EXPERIMENT_NAME": "LSTM_AE_Anomalias_ECG_v1",  # ⚠️ CAMBIA ESTE NOMBRE PARA CADA EXPERIMENTO
    "MLFLOW_TRACKING_URI": None,  # None = usa el directorio local
    
    # ===== RUTAS =====
    "OUTPUT_DIR": "./outputs",  # Directorio para guardar modelos y artefactos
}

# Crear directorio de salida
Path(CONFIG["OUTPUT_DIR"]).mkdir(parents=True, exist_ok=True)

print("✓ Configuración cargada:")
print(json.dumps(CONFIG, indent=2, ensure_ascii=False))


✓ Configuración cargada:
{
  "INPUT_SIZE": 3,
  "HIDDEN_SIZES_ENCODER": [
    64,
    32
  ],
  "HIDDEN_SIZES_DECODER": [
    32,
    64
  ],
  "LATENT_DIM": 16,
  "NUM_LAYERS": 2,
  "DROPOUT": 0.1,
  "EPOCHS": 30,
  "BATCH_SIZE": 64,
  "LEARNING_RATE": 0.001,
  "WEIGHT_DECAY": 1e-05,
  "CLIP_GRAD_NORM": 1.0,
  "THRESHOLD_PERCENTILE": 95.0,
  "MLFLOW_EXPERIMENT_NAME": "LSTM_AE_Anomalias_ECG_v1",
  "MLFLOW_TRACKING_URI": null,
  "OUTPUT_DIR": "./outputs"
}


---

## 2. 📂 Carga y Preparación de Datos


In [214]:
# ========================================
# TODO: COMPLETA ESTA FUNCIÓN CON TU LÓGICA DE CARGA DE DATOS
# ========================================
def load_raw_data() -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Carga los datos crudos y los divide en train/val/test.
    
    Returns:
        Tuple con:
        - X_train_norm: Array numpy de forma (N_train, T, INPUT_SIZE) con ejemplos NORMALES para entrenamiento
        - X_val_norm: Array numpy de forma (N_val, T, INPUT_SIZE) con ejemplos NORMALES para validación
        - X_val_anom: Array numpy de forma (N_val_anom, T, INPUT_SIZE) con ejemplos ANÓMALOS para validación
        - y_val: Array numpy de forma (N_val_norm + N_val_anom,) con etiquetas 0=normal, 1=anómalo para validación
        - X_test_norm: Array numpy de forma (N_test, T, INPUT_SIZE) con ejemplos NORMALES para test
        - X_test_anom: Array numpy de forma (N_test_anom, T, INPUT_SIZE) con ejemplos ANÓMALOS para test
        - y_test: Array numpy de forma (N_test_norm + N_test_anom,) con etiquetas 0=normal, 1=anómalo para test
    """
    # ========================================
    # TODO: REEMPLAZA ESTE CÓDIGO CON TU LÓGICA
    # ========================================
    
    # EJEMPLO: Datos dummy para testing (ELIMINA ESTO Y PON TU CÓDIGO)
    print("⚠️ ADVERTENCIA: Usando datos dummy. Reemplaza esta función con tu lógica de carga.")
    
    # Parámetros de ejemplo
    T = 5000  # Longitud de la secuencia temporal
    INPUT_SIZE = CONFIG["INPUT_SIZE"]
    
    # Generar datos dummy
    np.random.seed(SEED)
    
    # Train: solo normales (1000 ejemplos)
    X_train_norm = np.random.randn(1000, T, INPUT_SIZE).astype(np.float32)
    
    # Val: normales (200) + anómalos (50)
    X_val_norm = np.random.randn(200, T, INPUT_SIZE).astype(np.float32)
    X_val_anom = np.random.randn(50, T, INPUT_SIZE).astype(np.float32) + 2.0  # Anómalos con offset
    y_val = np.concatenate([np.zeros(len(X_val_norm)), np.ones(len(X_val_anom))]).astype(np.int64)
    
    # Test: normales (200) + anómalos (50)
    X_test_norm = np.random.randn(200, T, INPUT_SIZE).astype(np.float32)
    X_test_anom = np.random.randn(50, T, INPUT_SIZE).astype(np.float32) + 2.0  # Anómalos con offset
    y_test = np.concatenate([np.zeros(len(X_test_norm)), np.ones(len(X_test_anom))]).astype(np.int64)
    
    # ========================================
    # FIN DEL CÓDIGO DUMMY - PON TU LÓGICA AQUÍ
    # ========================================
    
    return X_train_norm, X_val_norm, X_val_anom, y_val, X_test_norm, X_test_anom, y_test


In [215]:
# ========================================
# Preparación de datos (PASA DIRECTAMENTE - sin modificación)
# ========================================
def normalize_data(
    X_train: np.ndarray,
    X_val_norm: np.ndarray,
    X_val_anom: np.ndarray,
    X_test_norm: np.ndarray,
    X_test_anom: np.ndarray,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, Dict[str, float]]:
    """
    Pasa los datos directamente sin modificarlos.
    Los datos de Datos_no_supervisados ya vienen procesados y listos para usar.
    
    Returns:
        Tupla con los mismos datos (sin modificar) y diccionario con estadísticas de referencia.
    """
    print("="*70)
    print("📊 PREPARACIÓN DE DATOS")
    print("="*70)
    print("✓ Los datos de 'Datos_no_supervisados' ya vienen procesados y listos.")
    print("  → Pasando datos directamente SIN modificaciones (sin limpieza ni normalización)")
    print("="*70)
    
    # Usar datos tal como están - SIN MODIFICACIONES
    X_train_norm = X_train
    X_val_norm_norm = X_val_norm
    X_val_anom_norm = X_val_anom
    X_test_norm_norm = X_test_norm
    X_test_anom_norm = X_test_anom
    
    # Calcular estadísticas solo para referencia/logging (no para normalizar)
    mean = np.mean(X_train, axis=(0, 1))
    std = np.std(X_train, axis=(0, 1))
    stats = {
        "mean": mean,
        "std": std,
        "normalized": False,
        "cleaned": False,
        "note": "Datos usados directamente sin modificación"
    }
    
    print(f"\n📈 Estadísticas de referencia (solo para logging):")
    print(f"  Mean: {mean}")
    print(f"  Std: {std}")
    print(f"\n✓ Datos listos para usar (sin modificaciones)")
    print("="*70)
    
    return X_train_norm, X_val_norm_norm, X_val_anom_norm, X_test_norm_norm, X_test_anom_norm, stats


---

## 3. 🧠 Definición del Modelo LSTM Autoencoder


In [216]:
# ========================================
# Clase LSTM Autoencoder
# ========================================
class LSTMAutoencoder(nn.Module):
    """
    Autoencoder basado en LSTM para detección de anomalías en series temporales.
    
    Arquitectura:
    - Encoder: LSTM que comprime la secuencia a un espacio latente
    - Decoder: LSTM que reconstruye la secuencia desde el espacio latente
    """
    
    def __init__(
        self,
        input_size: int,
        hidden_sizes_encoder: List[int],
        hidden_sizes_decoder: List[int],
        latent_dim: int,
        num_layers: int = 2,
        dropout: float = 0.1,
    ):
        super(LSTMAutoencoder, self).__init__()
        
        self.input_size = input_size
        self.latent_dim = latent_dim
        
        # Validar que num_layers coincida con las longitudes de hidden_sizes
        assert len(hidden_sizes_encoder) == num_layers, \
            f"len(hidden_sizes_encoder)={len(hidden_sizes_encoder)} debe ser igual a num_layers={num_layers}"
        assert len(hidden_sizes_decoder) == num_layers, \
            f"len(hidden_sizes_decoder)={len(hidden_sizes_decoder)} debe ser igual a num_layers={num_layers}"
        
        # ===== ENCODER =====
        encoder_layers = []
        for i in range(num_layers):
            in_size = input_size if i == 0 else hidden_sizes_encoder[i - 1]
            out_size = hidden_sizes_encoder[i]
            encoder_layers.append(
                nn.LSTM(
                    input_size=in_size,
                    hidden_size=out_size,
                    num_layers=1,
                    batch_first=True,
                    dropout=0.0 if i == num_layers - 1 else dropout,
                )
            )
        self.encoder = nn.ModuleList(encoder_layers)
        
        # Capa lineal para mapear al espacio latente
        self.latent_projection = nn.Linear(hidden_sizes_encoder[-1], latent_dim)
        
        # ===== DECODER =====
        # Capa para expandir desde el espacio latente
        self.latent_expansion = nn.Linear(latent_dim, hidden_sizes_decoder[0])
        
        decoder_layers = []
        for i in range(num_layers):
            in_size = hidden_sizes_decoder[i]
            out_size = hidden_sizes_decoder[i + 1] if i < num_layers - 1 else input_size
            decoder_layers.append(
                nn.LSTM(
                    input_size=in_size,
                    hidden_size=out_size,
                    num_layers=1,
                    batch_first=True,
                    dropout=0.0 if i == num_layers - 1 else dropout,
                )
            )
        self.decoder = nn.ModuleList(decoder_layers)
        
    def encode(self, x: torch.Tensor) -> torch.Tensor:
        """
        Codifica la secuencia de entrada al espacio latente.
        
        Args:
            x: Tensor de forma (batch_size, seq_len, input_size)
        
        Returns:
            Tensor latente de forma (batch_size, latent_dim)
        """
        h = x
        for lstm_layer in self.encoder:
            h, (hidden, cell) = lstm_layer(h)
            # Usar el último hidden state
            h = hidden[-1]  # (batch_size, hidden_size)
            h = h.unsqueeze(1)  # (batch_size, 1, hidden_size) para siguiente capa
        
        # Tomar el último hidden state de la última capa
        last_hidden = h.squeeze(1)  # (batch_size, hidden_size)
        
        # Proyectar al espacio latente
        latent = self.latent_projection(last_hidden)  # (batch_size, latent_dim)
        return latent
    
    def decode(self, latent: torch.Tensor, seq_len: int) -> torch.Tensor:
        """
        Decodifica desde el espacio latente a la secuencia reconstruida.
        
        Args:
            latent: Tensor latente de forma (batch_size, latent_dim)
            seq_len: Longitud de la secuencia a reconstruir
        
        Returns:
            Tensor reconstruido de forma (batch_size, seq_len, input_size)
        """
        # Expandir el espacio latente
        h = self.latent_expansion(latent)  # (batch_size, hidden_size_decoder[0])
        h = h.unsqueeze(1)  # (batch_size, 1, hidden_size_decoder[0])
        
        # Repetir para toda la secuencia
        h = h.repeat(1, seq_len, 1)  # (batch_size, seq_len, hidden_size_decoder[0])
        
        # Decodificar capa por capa
        for lstm_layer in self.decoder:
            h, (hidden, cell) = lstm_layer(h)
        
        return h  # (batch_size, seq_len, input_size)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass completo: encode -> decode.
        
        Args:
            x: Tensor de forma (batch_size, seq_len, input_size)
        
        Returns:
            Tensor reconstruido de forma (batch_size, seq_len, input_size)
        """
        latent = self.encode(x)
        seq_len = x.size(1)
        reconstructed = self.decode(latent, seq_len)
        return reconstructed


In [217]:
# ========================================
# Función para calcular error de reconstrucción
# ========================================
def compute_reconstruction_error(
    model: LSTMAutoencoder,
    dataloader: DataLoader,
    device: torch.device,
) -> np.ndarray:
    """
    Calcula el error de reconstrucción (MSE) para cada muestra en el dataloader.
    
    Returns:
        Array numpy con un error por muestra.
    """
    model.eval()
    errors = []
    
    with torch.no_grad():
        for batch in dataloader:
            # El dataloader devuelve una tupla con un tensor
            x = batch[0].to(device)  # (batch_size, seq_len, input_size)
            
            # Reconstruir
            x_recon = model(x)
            
            # Calcular MSE por muestra (promedio sobre timesteps y features)
            mse = torch.mean((x_recon - x) ** 2, dim=(1, 2))  # (batch_size,)
            errors.append(mse.cpu().numpy())
    
    return np.concatenate(errors)


In [218]:
# ========================================
# Instanciar modelo
# ========================================
def create_model(config: Dict) -> LSTMAutoencoder:
    """Crea e instancia el modelo LSTM Autoencoder."""
    model = LSTMAutoencoder(
        input_size=config["INPUT_SIZE"],
        hidden_sizes_encoder=config["HIDDEN_SIZES_ENCODER"],
        hidden_sizes_decoder=config["HIDDEN_SIZES_DECODER"],
        latent_dim=config["LATENT_DIM"],
        num_layers=config["NUM_LAYERS"],
        dropout=config["DROPOUT"],
    )
    
    # Contar parámetros
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"✓ Modelo creado:")
    print(f"  Parámetros totales: {total_params:,} ({total_params / 1e6:.2f}M)")
    print(f"  Parámetros entrenables: {trainable_params:,}")
    
    return model


---

## 4. 🏋️ Funciones de Entrenamiento y Evaluación


In [219]:
# ========================================
# Diagnóstico de errores de reconstrucción y umbral
# ========================================
def diagnosticar_errores_umbral(
    errors_norm: np.ndarray,
    errors_anom: np.ndarray,
    threshold: float,
    nombre: str = "DIAGNÓSTICO"
):
    """Muestra estadísticas de errores de reconstrucción y umbral."""
    print("\n" + "="*70)
    print(f"🔍 {nombre}: ERRORES DE RECONSTRUCCIÓN Y UMBRAL")
    print("="*70)
    
    if len(errors_norm) > 0:
        print(f"\n📊 Errores NORMALES ({len(errors_norm)} muestras):")
        print(f"  - Min: {errors_norm.min():.8f}")
        print(f"  - Max: {errors_norm.max():.8f}")
        print(f"  - Mean: {errors_norm.mean():.8f}")
        print(f"  - Median: {np.median(errors_norm):.8f}")
        print(f"  - Std: {errors_norm.std():.8f}")
        print(f"  - Percentil 95: {np.percentile(errors_norm, 95):.8f}")
        print(f"  - Percentil 99: {np.percentile(errors_norm, 99):.8f}")
    else:
        print(f"\n⚠️ No hay errores normales disponibles")
    
    if len(errors_anom) > 0:
        print(f"\n📊 Errores ANÓMALOS ({len(errors_anom)} muestras):")
        print(f"  - Min: {errors_anom.min():.8f}")
        print(f"  - Max: {errors_anom.max():.8f}")
        print(f"  - Mean: {errors_anom.mean():.8f}")
        print(f"  - Median: {np.median(errors_anom):.8f}")
        print(f"  - Std: {errors_anom.std():.8f}")
        print(f"  - Percentil 95: {np.percentile(errors_anom, 95):.8f}")
        print(f"  - Percentil 99: {np.percentile(errors_anom, 99):.8f}")
    else:
        print(f"\n⚠️ No hay errores anómalos disponibles")
    
    print(f"\n🎯 UMBRAL DE DETECCIÓN: {threshold:.8f}")
    
    if len(errors_norm) > 0 and len(errors_anom) > 0:
        # Predicciones esperadas
        normales_detectados = (errors_norm > threshold).sum()
        anomalos_detectados = (errors_anom > threshold).sum()
        
        print(f"\n📈 CLASIFICACIÓN CON ESTE UMBRAL:")
        print(f"  - Normales > umbral (predicho como anómalos): {normales_detectados}/{len(errors_norm)} ({100*normales_detectados/len(errors_norm):.2f}%)")
        print(f"  - Anómalos > umbral (predicho como anómalos): {anomalos_detectados}/{len(errors_anom)} ({100*anomalos_detectados/len(errors_anom):.2f}%)")
        
        # Separabilidad
        mean_norm = errors_norm.mean()
        mean_anom = errors_anom.mean()
        separabilidad = abs(mean_anom - mean_norm) / (errors_norm.std() + errors_anom.std() + 1e-8)
        
        print(f"\n📊 SEPARABILIDAD:")
        print(f"  - Diferencia de medias: {abs(mean_anom - mean_norm):.8f}")
        print(f"  - Separabilidad (Cohen's d aproximado): {separabilidad:.4f}")
        
        if separabilidad < 0.5:
            print(f"  ⚠️ ADVERTENCIA: Separabilidad muy baja (< 0.5). El modelo no está diferenciando bien normales vs anómalos.")
        if mean_anom <= mean_norm:
            print(f"  ❌ ERROR: Los anómalos tienen errores MENORES que los normales! El modelo está aprendiendo al revés.")
        
        # Sugerencias de umbral
        if threshold < errors_norm.max() and threshold < errors_anom.max():
            percentil_optimo = None
            for p in [90, 95, 98, 99]:
                umbral_candidato = np.percentile(errors_norm, p)
                tp = (errors_anom > umbral_candidato).sum()
                fp = (errors_norm > umbral_candidato).sum()
                fn = (errors_anom <= umbral_candidato).sum()
                tn = (errors_norm <= umbral_candidato).sum()
                
                precision = tp / (tp + fp) if (tp + fp) > 0 else 0
                recall = tp / (tp + fn) if (tp + fn) > 0 else 0
                f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
                
                if f1 > 0.3:  # Solo sugerir si tiene algo de F1
                    percentil_optimo = p
                    print(f"\n💡 SUGERENCIA: Percentil {p} da umbral {umbral_candidato:.8f} -> F1={f1:.4f}, Precision={precision:.4f}, Recall={recall:.4f}")
                    break
    
    print("="*70 + "\n")


In [220]:
# ========================================
# Función de entrenamiento por época
# ========================================
def train_one_epoch(
    model: LSTMAutoencoder,
    train_loader: DataLoader,
    optimizer: optim.Optimizer,
    device: torch.device,
    config: Dict,
) -> Tuple[float, float]:
    """
    Entrena el modelo por una época.
    
    Returns:
        Tupla con (loss_promedio, reconstruction_error_promedio)
    """
    model.train()
    total_loss = 0.0
    total_recon_error = 0.0
    n_samples = 0
    
    criterion = nn.MSELoss()
    
    for batch in train_loader:
        x = batch[0].to(device)  # (batch_size, seq_len, input_size)
        
        # Verificar que los datos sean finitos
        if not torch.isfinite(x).all():
            print("⚠️ ADVERTENCIA: Datos de entrada contienen valores no finitos (NaN/Inf)")
            continue
        
        optimizer.zero_grad()
        
        # Forward pass
        x_recon = model(x)
        
        # Verificar que la reconstrucción sea finita
        if not torch.isfinite(x_recon).all():
            print("⚠️ ADVERTENCIA: Reconstrucción contiene valores no finitos (NaN/Inf)")
            continue
        
        # Calcular pérdida (MSE)
        loss = criterion(x_recon, x)
        
        # Verificar que la pérdida sea finita
        if not torch.isfinite(loss):
            print(f"⚠️ ADVERTENCIA: Pérdida no finita: {loss.item()}")
            continue
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping (opcional)
        if config.get("CLIP_GRAD_NORM") is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), config["CLIP_GRAD_NORM"])
        
        optimizer.step()
        
        # Acumular métricas
        batch_size = x.size(0)
        loss_value = loss.item()
        total_loss += loss_value * batch_size
        
        # Error de reconstrucción promedio por muestra (MSE por muestra, luego promedio)
        # Calcular MSE por muestra: (batch_size,)
        mse_per_sample = torch.mean((x_recon - x) ** 2, dim=(1, 2))  # (batch_size,)
        # Promedio sobre todas las muestras del batch
        recon_error = mse_per_sample.mean().item()
        
        # Verificar que el error de reconstrucción sea finito
        if not np.isfinite(recon_error):
            print(f"⚠️ ADVERTENCIA: Error de reconstrucción no finito: {recon_error}")
            recon_error = loss_value  # Usar la pérdida como fallback
        
        total_recon_error += recon_error * batch_size
        n_samples += batch_size
    
    avg_loss = total_loss / n_samples if n_samples > 0 else 0.0
    avg_recon_error = total_recon_error / n_samples if n_samples > 0 else 0.0
    
    # Verificar que los promedios sean finitos
    if not np.isfinite(avg_loss):
        print(f"⚠️ ADVERTENCIA: Pérdida promedio no finita: {avg_loss}")
        avg_loss = 0.0
    
    if not np.isfinite(avg_recon_error):
        print(f"⚠️ ADVERTENCIA: Error de reconstrucción promedio no finito: {avg_recon_error}")
        avg_recon_error = avg_loss  # Usar la pérdida como fallback
    
    return avg_loss, avg_recon_error


In [221]:
# ========================================
# Función de validación (con métricas para ambas clases)
# ========================================
def validate(
    model: LSTMAutoencoder,
    val_norm_loader: DataLoader,
    val_anom_loader: DataLoader,
    y_val: np.ndarray,
    device: torch.device,
    threshold: float,
) -> Dict[str, float]:
    """
    Evalúa el modelo en el conjunto de validación.
    Calcula precision, recall y f1 para ambas clases (normal y anómalo).
    
    Returns:
        Diccionario con métricas completas y matriz de confusión
    """
    model.eval()
    
    # Calcular errores de reconstrucción
    errors_norm = compute_reconstruction_error(model, val_norm_loader, device)
    errors_anom = compute_reconstruction_error(model, val_anom_loader, device) if val_anom_loader else np.array([])
    
    # Combinar errores y etiquetas
    all_errors = np.concatenate([errors_norm, errors_anom]) if len(errors_anom) > 0 else errors_norm
    
    # Predicciones basadas en umbral
    y_pred = (all_errors > threshold).astype(int)
    
    # Calcular métricas generales
    accuracy = accuracy_score(y_val, y_pred)
    
    # Calcular métricas por clase usando classification_report
    report = classification_report(y_val, y_pred, target_names=["normal", "anomalo"], output_dict=True, zero_division=0)
    
    # Métricas para clase normal (0)
    metrics_normal = report.get("normal", {})
    precision_normal = metrics_normal.get("precision", 0.0)
    recall_normal = metrics_normal.get("recall", 0.0)
    f1_normal = metrics_normal.get("f1-score", 0.0)
    
    # Métricas para clase anómala (1)
    metrics_anom = report.get("anomalo", {})
    precision_anom = metrics_anom.get("precision", 0.0)
    recall_anom = metrics_anom.get("recall", 0.0)
    f1_anom = metrics_anom.get("f1-score", 0.0)
    
    # Métricas generales (macro avg)
    macro_avg = report.get("macro avg", {})
    precision_macro = macro_avg.get("precision", 0.0)
    recall_macro = macro_avg.get("recall", 0.0)
    f1_macro = macro_avg.get("f1-score", 0.0)
    
    # Matriz de confusión
    cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / max(1, tn + fp)  # TNR
    
    return {
        # Métricas generales
        "accuracy": accuracy,
        "specificity": specificity,
        # Métricas para clase normal
        "precision_normal": precision_normal,
        "recall_normal": recall_normal,
        "f1_normal": f1_normal,
        # Métricas para clase anómala
        "precision_anom": precision_anom,
        "recall_anom": recall_anom,
        "f1_anom": f1_anom,
        # Métricas macro promedio
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        # Matriz de confusión
        "confusion_matrix": cm,
        # Errores (para análisis)
        "errors_norm": errors_norm,
        "errors_anom": errors_anom,
    }


In [222]:
# ========================================
# Función de evaluación en test (con métricas para ambas clases)
# ========================================
def evaluate_on_test(
    model: LSTMAutoencoder,
    test_norm_loader: DataLoader,
    test_anom_loader: DataLoader,
    y_test: np.ndarray,
    device: torch.device,
    threshold: float,
) -> Dict[str, float]:
    """
    Evalúa el modelo en el conjunto de test.
    Calcula precision, recall y f1 para ambas clases (normal y anómalo).
    
    Returns:
        Diccionario con métricas completas y matriz de confusión
    """
    model.eval()
    
    # Calcular errores de reconstrucción
    errors_norm = compute_reconstruction_error(model, test_norm_loader, device)
    errors_anom = compute_reconstruction_error(model, test_anom_loader, device) if test_anom_loader else np.array([])
    
    # Combinar errores y etiquetas
    all_errors = np.concatenate([errors_norm, errors_anom]) if len(errors_anom) > 0 else errors_norm
    
    # Predicciones basadas en umbral
    y_pred = (all_errors > threshold).astype(int)
    
    # Calcular métricas generales
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calcular métricas por clase usando classification_report
    report = classification_report(y_test, y_pred, target_names=["normal", "anomalo"], output_dict=True, zero_division=0)
    
    # Métricas para clase normal (0)
    metrics_normal = report.get("normal", {})
    precision_normal = metrics_normal.get("precision", 0.0)
    recall_normal = metrics_normal.get("recall", 0.0)
    f1_normal = metrics_normal.get("f1-score", 0.0)
    
    # Métricas para clase anómala (1)
    metrics_anom = report.get("anomalo", {})
    precision_anom = metrics_anom.get("precision", 0.0)
    recall_anom = metrics_anom.get("recall", 0.0)
    f1_anom = metrics_anom.get("f1-score", 0.0)
    
    # Métricas generales (macro avg)
    macro_avg = report.get("macro avg", {})
    precision_macro = macro_avg.get("precision", 0.0)
    recall_macro = macro_avg.get("recall", 0.0)
    f1_macro = macro_avg.get("f1-score", 0.0)
    
    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    specificity = tn / max(1, tn + fp)  # TNR
    
    return {
        # Métricas generales
        "accuracy": accuracy,
        "specificity": specificity,
        # Métricas para clase normal
        "precision_normal": precision_normal,
        "recall_normal": recall_normal,
        "f1_normal": f1_normal,
        # Métricas para clase anómala
        "precision_anom": precision_anom,
        "recall_anom": recall_anom,
        "f1_anom": f1_anom,
        # Métricas macro promedio
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "f1_macro": f1_macro,
        # Matriz de confusión
        "confusion_matrix": cm,
        # Errores (para análisis)
        "errors_norm": errors_norm,
        "errors_anom": errors_anom,
    }


---

## 5. 📊 Integración con MLflow


In [223]:
# ========================================
# Configuración de MLflow
# ========================================
def setup_mlflow(config: Dict) -> str:
    """
    Configura MLflow y crea/obtiene el experimento.
    
    Returns:
        ID del experimento
    """
    # Configurar tracking URI (similar al notebook de referencia)
    if config.get("MLFLOW_TRACKING_URI") is not None:
        mlflow.set_tracking_uri(config["MLFLOW_TRACKING_URI"])
    else:
        # Usar sqlite en el directorio padre (como en el notebook de referencia)
        PARENT_DIR = Path.cwd().parent.resolve()
        TRACKING_DB = (PARENT_DIR / "mlflow.db").resolve()
        mlflow.set_tracking_uri(f"sqlite:///{TRACKING_DB.as_posix()}")
        print(f"✓ MLflow tracking URI: sqlite:///{TRACKING_DB.as_posix()}")
    
    # Crear o obtener experimento
    experiment_name = config["MLFLOW_EXPERIMENT_NAME"]
    
    try:
        experiment = mlflow.get_experiment_by_name(experiment_name)
        if experiment is None:
            # Crear directorio de artefactos
            PARENT_DIR = Path.cwd().parent.resolve()
            ARTIFACT_ROOT = (PARENT_DIR / "mlflow_artifacts").resolve()
            ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)
            experiment_id = mlflow.create_experiment(experiment_name, artifact_location=ARTIFACT_ROOT.as_uri())
            print(f"✓ Experimento MLflow creado: {experiment_name} (ID: {experiment_id})")
            print(f"  Artifact root: {ARTIFACT_ROOT.as_uri()}")
        else:
            experiment_id = experiment.experiment_id
            print(f"✓ Experimento MLflow existente: {experiment_name} (ID: {experiment_id})")
    except Exception as e:
        print(f"⚠ Error al configurar MLflow: {e}")
        experiment_id = mlflow.set_experiment(experiment_name)
    
    return experiment_id


In [224]:
# ========================================
# Función para guardar matriz de confusión como artefacto
# ========================================
def save_confusion_matrix(
    cm: np.ndarray,
    output_dir: Path,
    tag: str,
) -> Tuple[Path, Path]:
    """
    Guarda la matriz de confusión como PNG y CSV.
    
    Returns:
        Tupla con rutas (png_path, csv_path)
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Guardar como CSV
    csv_path = output_dir / f"confusion_matrix_{tag}.csv"
    df_cm = pd.DataFrame(cm, index=["Normal", "Anómalo"], columns=["Normal", "Anómalo"])
    df_cm.to_csv(csv_path)
    
    # Guardar como PNG
    png_path = output_dir / f"confusion_matrix_{tag}.png"
    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(cm, interpolation="nearest", cmap="Blues")
    ax.figure.colorbar(im, ax=ax)
    
    # Etiquetas
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]))
    ax.set_xticklabels(["Normal", "Anómalo"])
    ax.set_yticklabels(["Normal", "Anómalo"])
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Real")
    ax.set_title(f"Matriz de Confusión - {tag.upper()}")
    
    # Añadir valores en las celdas
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j, i, f"{cm[i, j]}",
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black"
            )
    
    plt.tight_layout()
    plt.savefig(png_path, dpi=150)
    plt.close()
    
    return png_path, csv_path


In [225]:
# ========================================
# Función para guardar gráficos de curvas de entrenamiento
# ========================================
def save_training_curves(
    train_losses: List[float],
    train_recon_errors: List[float],
    val_f1_scores: List[float],
    output_dir: Path,
) -> Path:
    """
    Guarda gráficos de curvas de entrenamiento.
    
    Returns:
        Ruta del archivo PNG guardado
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    epochs = range(1, len(train_losses) + 1)
    
    # Loss
    axes[0].plot(epochs, train_losses, label="Train Loss", color="blue")
    axes[0].set_xlabel("Época")
    axes[0].set_ylabel("Loss (MSE)")
    axes[0].set_title("Loss de Entrenamiento")
    axes[0].grid(True, alpha=0.3)
    axes[0].legend()
    
    # Reconstruction Error
    axes[1].plot(epochs, train_recon_errors, label="Train Recon Error", color="green")
    axes[1].set_xlabel("Época")
    axes[1].set_ylabel("Error de Reconstrucción")
    axes[1].set_title("Error de Reconstrucción")
    axes[1].grid(True, alpha=0.3)
    axes[1].legend()
    
    # F1 Score en validación
    if val_f1_scores:
        axes[2].plot(epochs, val_f1_scores, label="Val F1", color="red")
        axes[2].set_xlabel("Época")
        axes[2].set_ylabel("F1-Score")
        axes[2].set_title("F1-Score en Validación")
        axes[2].grid(True, alpha=0.3)
        axes[2].legend()
    
    plt.tight_layout()
    
    png_path = output_dir / "training_curves.png"
    plt.savefig(png_path, dpi=150)
    plt.close()
    
    return png_path


---

## 6. 🪄 Orquestación con Prefect


In [ ]:
# ========================================

# Flujo principal de Prefect
# ========================================
@flow(name="lstm_autoencoder_training_flow", log_prints=True)
def lstm_autoencoder_training_flow(config: Dict = None):
    """
    Flujo principal de Prefect que orquesta todo el proceso:
    1. Carga y preparación de datos
    2. Creación del modelo
    3. Entrenamiento
    4. Evaluación en test
    """
    if config is None:
        config = CONFIG
    
    print("🚀 Iniciando flujo de entrenamiento LSTM Autoencoder...")
    print(f"Experimento MLflow: {config['MLFLOW_EXPERIMENT_NAME']}")
    
    # Configurar MLflow
    experiment_id = setup_mlflow(config)
    
    # Cargar y preparar datos
    dataloaders = task_load_data(config)
    train_loader, val_norm_loader, val_anom_loader, y_val, test_norm_loader, test_anom_loader, y_test = dataloaders
    
    # Crear modelo
    print("🧠 Creando modelo...")
    model = create_model(config)
    
    # Entrenar
    model, threshold, train_losses, train_recon_errors, val_f1_scores, best_f1 = task_train_model(
        model, train_loader, val_norm_loader, val_anom_loader, y_val, config, DEVICE, experiment_id
    )
    
    # Evaluar en test
    test_metrics = task_evaluate_test(
        model, test_norm_loader, test_anom_loader, y_test, DEVICE, threshold, config, experiment_id
    )
    
    print("\n" + "="*60)
    print("✅ FLUJO COMPLETADO")
    print("="*60)
    print(f"Mejor F1 en validación: {best_f1:.4f}")
    print(f"F1 en test: {test_metrics['f1']:.4f}")
    print(f"Umbral usado: {threshold:.6f}")
    print(f"\nRevisa MLflow para ver todos los artefactos y métricas.")
    
    return {
        "model": model,
        "threshold": threshold,
        "test_metrics": test_metrics,
        "best_f1": best_f1,
    }


In [227]:
# ========================================
# Ejecutar el flujo completo
# ========================================
if __name__ == "__main__":
    results = lstm_autoencoder_training_flow(CONFIG)
    print("\n✓ Proceso finalizado exitosamente")


2025-11-21 22:36:49 INFO  [prefect.flow_runs] Beginning flow run 'imaginary-macaque' for flow 'lstm_autoencoder_training_flow'
2025-11-21 22:36:49 INFO  [prefect.flow_runs] 🚀 Iniciando flujo de entrenamiento LSTM Autoencoder...
2025-11-21 22:36:49 INFO  [prefect.flow_runs] Experimento MLflow: LSTM_AE_Anomalias_ECG_v1
2025-11-21 22:36:49 INFO  [prefect.flow_runs] ✓ MLflow tracking URI: sqlite:///S:/Proyecto final/mlflow.db
2025-11-21 22:36:49 INFO  [prefect.flow_runs] ✓ Experimento MLflow existente: LSTM_AE_Anomalias_ECG_v1 (ID: 3)
2025-11-21 22:36:49 INFO  [prefect.task_runs] 📂 Cargando datos...
2025-11-21 22:36:49 INFO  [prefect.task_runs] ⚠️ ADVERTENCIA: Usando datos dummy. Reemplaza esta función con tu lógica de carga.


2025-11-21 22:36:49 INFO  [prefect.task_runs] ======================================================================
2025-11-21 22:36:49 INFO  [prefect.task_runs] 📊 PREPARACIÓN DE DATOS
2025-11-21 22:36:49 INFO  [prefect.task_runs] ======================================================================
2025-11-21 22:36:49 INFO  [prefect.task_runs] ✓ Los datos de 'Datos_no_supervisados' ya vienen procesados y listos.
2025-11-21 22:36:49 INFO  [prefect.task_runs]   → Pasando datos directamente SIN modificaciones (sin limpieza ni normalización)
2025-11-21 22:36:49 INFO  [prefect.task_runs] ======================================================================
2025-11-21 22:36:50 INFO  [prefect.task_runs] 
📈 Estadísticas de referencia (solo para logging):
2025-11-21 22:36:50 INFO  [prefect.task_runs]   Mean: [ 1.6704485e-04 -1.6632090e-04 -8.0895676e-05]
2025-11-21 22:36:50 INFO  [prefect.task_runs]   Std: [0.99752015 0.996813   0.9971317 ]
2025-11-21 22:36:50 INFO  [prefect.task_runs] 
✓ D

KeyError: 'f1'

---

## ✅ Checklist Final

Antes de ejecutar el notebook completo:

1. ✅ **Completa la función `load_raw_data()`** con tu lógica de carga de datos
2. ✅ **Ajusta los hiperparámetros** en la sección de configuración (CONFIG)
3. ✅ **Cambia el nombre del experimento MLflow** (`MLFLOW_EXPERIMENT_NAME`)
4. ✅ **Verifica que los datos tengan la forma correcta**: (N, T, INPUT_SIZE)
5. ✅ **Ejecuta todas las celdas en orden**

### 📝 Notas importantes:

- El modelo se entrena **solo con ejemplos normales** (entrenamiento no supervisado)
- El umbral de detección se calcula automáticamente usando el percentil especificado en `THRESHOLD_PERCENTILE`
- Todos los artefactos (modelo, gráficos, matrices de confusión) se guardan en `OUTPUT_DIR` y en MLflow
- Las métricas se registran en MLflow: `train_loss`, `train_reconstruction_error`, `val_precision`, `val_recall`, `val_f1`, `test_precision`, `test_recall`, `test_f1`
